# Elo Editorial Group - Editorial Assistant Demo

This notebook demonstrates the **Editorial Assistant** system built with **CrewAI** and **Google's Gemini LLM**.

## 📚 Overview

The Editorial Assistant is a multiagent system that helps customers:
- 📖 Find detailed book information
- 🏪 Locate stores selling specific books
- 🎫 Create support tickets

## 🏗️ Architecture

- **Orchestrator Agent**: Detects user intent and coordinates responses
- **Catalog Agent**: Handles book searches and store locations
- **Support Agent**: Manages customer service and tickets

## 🛠️ Setup Instructions

1. **Install dependencies**:
   ```bash
   pip install crewai google-generativeai langchain-google-genai python-dotenv fastapi uvicorn
   ```

2. **Get a Gemini API key**:
   - Visit [Google AI Studio](https://makersuite.google.com/app/apikey)
   - Create a new API key (free tier available)

3. **Set environment variables**:
   ```python
   import os
   os.environ['GEMINI_API_KEY'] = 'your_api_key_here'
   os.environ['GEMINI_MODEL'] = 'gemini-1.5-flash'
   ```

## 🔧 Environment Setup

In [ ]:
# Install required packages (run this cell if packages are not installed)
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

packages = [
    "crewai>=0.36.0",
    "google-generativeai>=0.7.0", 
    "langchain-google-genai>=1.0.0",
    "python-dotenv>=1.0.0",
    "fastapi>=0.104.0",
    "uvicorn>=0.24.0"
]

print("Installing packages...")
for package in packages:
    try:
        install_package(package)
        print(f"✅ {package} installed successfully")
    except Exception as e:
        print(f"❌ Failed to install {package}: {e}")

print("\n🎉 Installation complete!")

In [ ]:
# Set up environment variables
import os
from getpass import getpass

# Get API key from user (this will be hidden when typed)
api_key = getpass("Enter your Gemini API key: ")
os.environ['GEMINI_API_KEY'] = api_key
os.environ['GEMINI_MODEL'] = 'gemini-1.5-flash'

print("✅ Environment variables set successfully!")

## 📊 Sample Data

Let's examine the book catalog and create sample data for demonstration.

In [ ]:
# Create sample catalog data
import json

# Sample book catalog (subset of the full catalog)
sample_catalog = {
    "books": [
        {
            "title": "A Abelha",
            "author": "Milton Célio de Oliveira Filho",
            "imprint": "Elo Editora",
            "release_date": "15/04/2022",
            "synopsis": "Uma obra delicada que explora o universo das abelhas e sua importância para a natureza. Com ilustrações de Olavo Costa, o livro é uma jornada poética e educativa.",
            "availability": {
                "São Paulo": ["Livraria da Vila", "Livraria Cultura"],
                "Rio de Janeiro": ["Livraria Travessa"],
                "Online": ["Amazon.com.br", "Loja Elo Editorial"]
            }
        },
        {
            "title": "A Baleia-azul",
            "author": "Theo de Oliveira",
            "imprint": "Perabook",
            "release_date": "10/09/2023",
            "synopsis": "Acompanhe a vida majestosa do maior animal do planeta. Com ilustrações de Felipe Tognoli, esta obra mergulha nos mistérios dos oceanos de forma cativante.",
            "availability": {
                "São Paulo": ["Blooks Livraria"],
                "Online": ["Amazon.com.br", "Magazine Luiza", "Submarino"]
            }
        },
        {
            "title": "A Borboleta",
            "author": "Theo de Oliveira",
            "imprint": "Perabook",
            "release_date": "12/02/2024",
            "synopsis": "Celebra a beleza e a transformação da vida através do ciclo da borboleta. As ilustrações de Rosana Ferreira dão cor e vida a esta delicada narrativa.",
            "availability": {
                "Online": ["Amazon.com.br", "Loja Elo Editorial"]
            }
        }
    ]
}

# Create data directory and save catalog
import os
os.makedirs("data", exist_ok=True)

with open("data/mock_catalog.json", "w", encoding="utf-8") as f:
    json.dump(sample_catalog, f, indent=2, ensure_ascii=False)

# Initialize empty tickets file
with open("data/mock_tickets.json", "w", encoding="utf-8") as f:
    json.dump([], f)

print("📚 Sample data created successfully!")
print(f"📖 Books in catalog: {len(sample_catalog['books'])}")
for book in sample_catalog['books']:
    print(f"   • {book['title']} by {book['author']}")

## 🛠️ Core Tools Implementation

These are the fundamental tools that power the editorial assistant.

In [ ]:
# Editorial Tools Implementation
import json
import os
from datetime import datetime
from typing import Dict, Any, Optional

def get_book_details(book_title: str) -> Dict[str, Any]:
    """
    Retrieves book details from the catalog by title.
    
    Args:
        book_title (str): The title of the book to search for
        
    Returns:
        Dict containing book information
    """
    try:
        with open("data/mock_catalog.json", 'r', encoding='utf-8') as file:
            catalog_data = json.load(file)
        
        # Search for the book (case-insensitive)
        book_title_lower = book_title.lower()
        for book in catalog_data["books"]:
            if book_title_lower in book["title"].lower():
                return book
        
        return {"error": f"Book '{book_title}' not found in catalog"}
        
    except Exception as e:
        return {"error": f"Error: {str(e)}"}


def find_stores_selling_book(book_title: str, city: Optional[str] = None) -> Dict[str, Any]:
    """
    Finds stores selling a specific book, optionally filtered by city.
    
    Args:
        book_title (str): The title of the book to search for
        city (str, optional): The city to filter stores by
        
    Returns:
        Dict containing availability information
    """
    book_details = get_book_details(book_title)
    
    if "error" in book_details:
        return book_details
    
    availability = book_details.get("availability", {})
    
    if city:
        # Search for the city (case-insensitive)
        city_lower = city.lower()
        city_found = None
        
        for available_city in availability.keys():
            if city_lower in available_city.lower():
                city_found = available_city
                break
        
        if city_found:
            result = {
                "book_title": book_details["title"],
                "city": city_found,
                "stores": availability[city_found]
            }
            # Always include online stores
            if "Online" in availability:
                result["online_stores"] = availability["Online"]
            return result
        else:
            # City not found, return only online stores if available
            if "Online" in availability:
                return {
                    "book_title": book_details["title"],
                    "message": f"Book not available in physical stores in {city}",
                    "online_stores": availability["Online"]
                }
    else:
        # Return all availability information
        return {
            "book_title": book_details["title"],
            "availability": availability
        }


def open_support_ticket(name: str, email: str, subject: str, message: str) -> Dict[str, Any]:
    """
    Opens a new support ticket.
    
    Args:
        name (str): Customer's name
        email (str): Customer's email
        subject (str): Ticket subject
        message (str): Ticket message/description
        
    Returns:
        Dict containing ticket information
    """
    try:
        # Load existing tickets
        tickets = []
        try:
            with open("data/mock_tickets.json", 'r', encoding='utf-8') as file:
                tickets = json.load(file)
        except (FileNotFoundError, json.JSONDecodeError):
            tickets = []
        
        # Generate ticket ID
        ticket_id = f"TCK-{len(tickets) + 1:04d}"
        
        # Create new ticket
        new_ticket = {
            "id": ticket_id,
            "name": name,
            "email": email,
            "subject": subject,
            "message": message,
            "timestamp": datetime.now().isoformat(),
            "status": "open"
        }
        
        # Add to tickets list
        tickets.append(new_ticket)
        
        # Save back to file
        with open("data/mock_tickets.json", 'w', encoding='utf-8') as file:
            json.dump(tickets, file, indent=2, ensure_ascii=False)
        
        return {
            "ticket_id": ticket_id,
            "status": "open",
            "message": f"Support ticket {ticket_id} created successfully"
        }
        
    except Exception as e:
        return {"error": f"Failed to create ticket: {str(e)}"}

print("✅ Editorial tools implemented successfully!")

## 🧪 Tool Testing

Let's test each tool to ensure they work correctly.

In [ ]:
# Test book search functionality
print("🔍 TESTING BOOK SEARCH")
print("=" * 40)

# Test 1: Exact match
result = get_book_details("A Abelha")
print("\n1. Searching for 'A Abelha':")
print(f"   Title: {result.get('title', 'Not found')}")
print(f"   Author: {result.get('author', 'N/A')}")
print(f"   Synopsis: {result.get('synopsis', 'N/A')[:100]}...")

# Test 2: Partial match
result = get_book_details("Baleia")
print("\n2. Searching for 'Baleia' (partial match):")
print(f"   Title: {result.get('title', 'Not found')}")
print(f"   Author: {result.get('author', 'N/A')}")

# Test 3: Book not found
result = get_book_details("Nonexistent Book")
print("\n3. Searching for 'Nonexistent Book':")
print(f"   Result: {result}")

In [ ]:
# Test store finder functionality
print("🏪 TESTING STORE FINDER")
print("=" * 40)

# Test 1: All stores for a book
result = find_stores_selling_book("A Abelha")
print("\n1. All stores for 'A Abelha':")
print(f"   Book: {result.get('book_title')}")
availability = result.get('availability', {})
for city, stores in availability.items():
    print(f"   {city}: {', '.join(stores)}")

# Test 2: Stores in specific city
result = find_stores_selling_book("A Abelha", "São Paulo")
print("\n2. Stores for 'A Abelha' in São Paulo:")
print(f"   Physical stores: {result.get('stores', [])}")
print(f"   Online stores: {result.get('online_stores', [])}")

# Test 3: City without the book
result = find_stores_selling_book("A Abelha", "Salvador")
print("\n3. Stores for 'A Abelha' in Salvador:")
print(f"   Message: {result.get('message', 'N/A')}")
print(f"   Online alternatives: {result.get('online_stores', [])}")

# Test 4: Online-only book
result = find_stores_selling_book("A Borboleta")
print("\n4. Stores for 'A Borboleta' (online only):")
availability = result.get('availability', {})
for city, stores in availability.items():
    print(f"   {city}: {', '.join(stores)}")

In [ ]:
# Test support ticket functionality
print("🎫 TESTING SUPPORT TICKET SYSTEM")
print("=" * 40)

# Test 1: Create first ticket
result = open_support_ticket(
    name="Maria Silva",
    email="maria.silva@email.com",
    subject="Book submission inquiry",
    message="I would like to submit a manuscript for children's books. What is the process?"
)
print("\n1. Creating first support ticket:")
print(f"   Ticket ID: {result.get('ticket_id')}")
print(f"   Status: {result.get('status')}")
print(f"   Message: {result.get('message')}")

# Test 2: Create second ticket
result = open_support_ticket(
    name="João Santos",
    email="joao.santos@email.com",
    subject="Store availability question",
    message="Why is 'A Abelha' not available in my city (Brasília)?"
)
print("\n2. Creating second support ticket:")
print(f"   Ticket ID: {result.get('ticket_id')}")
print(f"   Status: {result.get('status')}")
print(f"   Message: {result.get('message')}")

# View all tickets
print("\n3. All support tickets:")
try:
    with open("data/mock_tickets.json", 'r', encoding='utf-8') as f:
        tickets = json.load(f)
    
    for ticket in tickets:
        print(f"   • {ticket['id']}: {ticket['subject']} ({ticket['name']})")
except Exception as e:
    print(f"   Error reading tickets: {e}")

## 🤖 CrewAI Integration

Now let's set up the full CrewAI system with agents and tasks.

In [ ]:
# Import CrewAI and related libraries
try:
    from crewai import Agent, Task, Crew
    from crewai.tools import tool
    from langchain_google_genai import ChatGoogleGenerativeAI
    
    print("✅ CrewAI libraries imported successfully!")
    
    # Initialize the LLM
    llm = ChatGoogleGenerativeAI(
        model=os.environ['GEMINI_MODEL'],
        google_api_key=os.environ['GEMINI_API_KEY'],
        temperature=0.7,
        max_tokens=1000
    )
    
    print("✅ Gemini LLM initialized successfully!")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please run the installation cell above to install required packages.")
except Exception as e:
    print(f"❌ Error initializing LLM: {e}")
    print("Please check your API key configuration.")

In [ ]:
# Define CrewAI tools
@tool("get_book_details")
def get_book_details_tool(book_title: str) -> str:
    """Get detailed information about a specific book from the catalog."""
    result = get_book_details(book_title)
    return json.dumps(result, ensure_ascii=False, indent=2)

@tool("find_stores_selling_book")
def find_stores_tool(book_title: str, city: str = None) -> str:
    """Find stores selling a specific book, optionally filtered by city."""
    result = find_stores_selling_book(book_title, city)
    return json.dumps(result, ensure_ascii=False, indent=2)

@tool("open_support_ticket")
def open_ticket_tool(name: str, email: str, subject: str, message: str) -> str:
    """Create a new support ticket with customer information."""
    result = open_support_ticket(name, email, subject, message)
    return json.dumps(result, ensure_ascii=False, indent=2)

print("✅ CrewAI tools defined successfully!")

In [ ]:
# Create CrewAI Agents

# Orchestrator Agent
orchestrator_agent = Agent(
    role="Editorial Assistant Orchestrator",
    goal="Detect user intent and coordinate with specialized agents to provide comprehensive editorial assistance",
    backstory="""You are the main coordinator of the Elo Editorial Group's digital assistant. 
    Your role is to understand what users need and coordinate with specialized agents to help them.
    You are knowledgeable about the publishing industry and can handle various customer inquiries.
    
    Always maintain a professional, helpful, and friendly tone.""",
    verbose=True,
    allow_delegation=True,
    llm=llm
)

# Catalog Agent
catalog_agent = Agent(
    role="Catalog and Commercial Specialist",
    goal="Provide detailed book information and help customers find where to purchase books",
    backstory="""You are a specialist in the Elo Editorial Group's catalog. You have deep knowledge 
    of all books, authors, and their availability across different stores and cities.
    
    Your expertise includes:
    - Book details (titles, authors, synopses, release dates)
    - Store availability across different cities
    - Online and physical store recommendations
    
    You provide accurate, detailed information and always try to help customers find 
    the best purchasing options for their needs.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[get_book_details_tool, find_stores_tool]
)

# Support Agent
support_agent = Agent(
    role="Customer Support and Author Relations Specialist",
    goal="Handle customer support inquiries and manage submission processes for authors",
    backstory="""You are a customer support specialist for the Elo Editorial Group. 
    You handle various customer inquiries, author submissions, and general support requests.
    
    Your responsibilities include:
    - Answering questions about submissions and publishing processes
    - Creating support tickets for customer issues
    - Providing information about author relations
    - Handling general customer service inquiries
    
    You are empathetic, patient, and always strive to provide excellent customer service.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[open_ticket_tool]
)

print("✅ CrewAI agents created successfully!")
print(f"   • Orchestrator Agent: {orchestrator_agent.role}")
print(f"   • Catalog Agent: {catalog_agent.role}")
print(f"   • Support Agent: {support_agent.role}")

## 🎯 Interactive Demo

Now let's test the complete system with realistic user interactions.

In [ ]:
# Demo 1: Book Information Request
print("📖 DEMO 1: Book Information Request")
print("=" * 50)

task = Task(
    description="""Find comprehensive details about the book titled 'A Abelha'.
    
    Provide:
    1. Complete book information (title, author, imprint, release date, synopsis)
    2. A clear, engaging description of the book
    3. Availability information
    
    Present the information in a friendly, informative manner.""",
    agent=catalog_agent,
    expected_output="A comprehensive description of the book with all catalog details"
)

crew = Crew(
    agents=[catalog_agent],
    tasks=[task],
    verbose=True
)

print("\nUser: Tell me about 'A Abelha'")
print("\nAssistant:")
result = crew.kickoff()
print(result)

In [ ]:
# Demo 2: Store Finder Request
print("\n\n🏪 DEMO 2: Store Finder Request")
print("=" * 50)

task = Task(
    description="""Help a customer find where to buy 'A Baleia-azul' in São Paulo.
    
    Provide:
    1. Physical stores in São Paulo selling the book
    2. Online purchasing options
    3. Clear, organized information
    
    If the book is not available in São Paulo, suggest online alternatives.""",
    agent=catalog_agent,
    expected_output="A detailed list of purchasing options including store names and locations"
)

crew = Crew(
    agents=[catalog_agent],
    tasks=[task],
    verbose=True
)

print("\nUser: Where can I buy 'A Baleia-azul' in São Paulo?")
print("\nAssistant:")
result = crew.kickoff()
print(result)

In [ ]:
# Demo 3: Support Ticket Creation
print("\n\n🎫 DEMO 3: Support Ticket Creation")
print("=" * 50)

task = Task(
    description="""Create a support ticket for a customer with the following details:
    - Name: Ana Costa
    - Email: ana.costa@email.com
    - Subject: Question about book submissions
    - Message: I'm an aspiring author and would like to know about the submission process for children's books. What are the requirements and timeline?
    
    After creating the ticket:
    1. Confirm the ticket creation
    2. Provide the ticket ID
    3. Explain next steps
    4. Offer additional assistance""",
    agent=support_agent,
    expected_output="Confirmation message with ticket details and follow-up instructions"
)

crew = Crew(
    agents=[support_agent],
    tasks=[task],
    verbose=True
)

print("\nUser: I need help with book submissions. Can you create a support ticket?")
print("\nAssistant:")
result = crew.kickoff()
print(result)

In [ ]:
# Demo 4: Complex Multi-Agent Interaction
print("\n\n🤖 DEMO 4: Complex Multi-Agent Interaction")
print("=" * 50)

task = Task(
    description="""A customer is asking: "I'm looking for books by Theo de Oliveira, and I'd like to know where I can buy them in Rio de Janeiro. Also, if there are any issues with availability, I'd like to open a support ticket."
    
    Coordinate with the appropriate agents to:
    1. Find books by Theo de Oliveira
    2. Check their availability in Rio de Janeiro
    3. Provide comprehensive purchasing information
    4. If needed, help with support ticket creation
    
    Provide a comprehensive, well-organized response.""",
    agent=orchestrator_agent,
    expected_output="A comprehensive response addressing all aspects of the customer's request"
)

crew = Crew(
    agents=[orchestrator_agent, catalog_agent, support_agent],
    tasks=[task],
    verbose=True
)

print("\nUser: I'm looking for books by Theo de Oliveira, and I'd like to know where I can buy them in Rio de Janeiro. Also, if there are any issues with availability, I'd like to open a support ticket.")
print("\nAssistant:")
result = crew.kickoff()
print(result)

## 📊 System Status & Summary

Let's check the current state of our system and summarize what we've accomplished.

In [ ]:
# System Status Check
print("📊 SYSTEM STATUS SUMMARY")
print("=" * 50)

# Check catalog
try:
    with open("data/mock_catalog.json", 'r', encoding='utf-8') as f:
        catalog = json.load(f)
    print(f"✅ Catalog: {len(catalog['books'])} books available")
except Exception as e:
    print(f"❌ Catalog: Error reading - {e}")

# Check tickets
try:
    with open("data/mock_tickets.json", 'r', encoding='utf-8') as f:
        tickets = json.load(f)
    print(f"✅ Support Tickets: {len(tickets)} tickets created")
    for ticket in tickets:
        print(f"   • {ticket['id']}: {ticket['subject']} ({ticket['status']})")
except Exception as e:
    print(f"❌ Support Tickets: Error reading - {e}")

# Check agents
try:
    print(f"✅ Agents: 3 agents active")
    print(f"   • {orchestrator_agent.role}")
    print(f"   • {catalog_agent.role}")
    print(f"   • {support_agent.role}")
except Exception as e:
    print(f"❌ Agents: Error - {e}")

# Check tools
print(f"✅ Tools: 3 tools implemented")
print(f"   • get_book_details")
print(f"   • find_stores_selling_book")
print(f"   • open_support_ticket")

print("\n🎉 Editorial Assistant System is fully operational!")

## 🚀 Next Steps

### Production Deployment

To deploy this system in production:

1. **API Development**: Use the FastAPI implementation provided in `api.py`
2. **Database Integration**: Replace JSON files with a proper database (PostgreSQL, MongoDB)
3. **Authentication**: Add user authentication and session management
4. **Rate Limiting**: Implement API rate limiting to prevent abuse
5. **Monitoring**: Add logging, metrics, and health checks
6. **Scalability**: Deploy with container orchestration (Docker + Kubernetes)

### Feature Enhancements

- **Advanced Search**: Implement fuzzy search and recommendations
- **Multi-language Support**: Add support for multiple languages
- **Voice Interface**: Integrate voice recognition and synthesis
- **Analytics**: Track user interactions and preferences
- **Personalization**: Provide personalized book recommendations

### Technical Improvements

- **Caching**: Implement Redis for faster response times
- **Error Handling**: Improve error handling and user feedback
- **Testing**: Add comprehensive unit and integration tests
- **Documentation**: Generate API documentation with OpenAPI/Swagger

## 🎯 Conclusion

This Editorial Assistant demonstrates a complete multiagent system built with CrewAI and Gemini LLM that can:

✅ **Search and retrieve detailed book information**  
✅ **Find stores selling specific books with location filtering**  
✅ **Create and manage customer support tickets**  
✅ **Coordinate multiple specialized agents for complex queries**  
✅ **Maintain conversation context and provide helpful responses**  

The system showcases clean architecture, proper separation of concerns, and robust error handling while maintaining an excellent user experience.

**Ready for production deployment with proper infrastructure and security measures!** 🚀